In [ ]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from preprocessamento import preprocessamento

csv_file = 'Datasets_csv/df_PUC.csv'
train, teste, val, _, _, _ = preprocessamento(csv_file, 0.6, 0.2, 0.2, True)

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.preprocessing import image
from keras.utils import array_to_img


In [ ]:
x_train, _ = next(train)
x_test, _ = next(teste)
x_val, _ = next(val)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

In [ ]:
from keras.utils import array_to_img
import matplotlib.pyplot as plt

def show_car_data(X, n=10, title=""):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        plt.imshow(array_to_img(X[i]))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.suptitle(title, fontsize=20)
    plt.show()


In [ ]:
show_car_data(x_train, title="Treino")
show_car_data(x_test, title="Teste")

In [ ]:
input_layer = Input(shape=(256, 256, 3), name="INPUT")
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)

code_layer = MaxPooling2D((2, 2), name="CODE")(x)

x = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(code_layer)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
output_layer = Conv2D(3, (3, 3), padding='same', name="OUTPUT")(x)

In [ ]:
AE = Model(input_layer, output_layer)
AE.compile(optimizer='adam', loss='mse')
AE.summary()

In [ ]:
AE.fit(x_train, x_train,code_layer = MaxPooling2D((2, 2), name="CODE")(x)

                epochs=10,
                batch_size=8,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
AE.save("autoencoder.h5")


In [ ]:
get_encoded = Model(inputs=AE.input, outputs=AE.get_layer("CODE").output)

In [ ]:
encoded_cat = get_encoded.predict(x_test)
encoded_cat = encoded_cat.reshape((len(x_test), 32*32*8)) #ver na code layer 
encoded_cat.shape

In [ ]:
reconstructed = AE.predict(x_test)


In [ ]:
def show_data(X, n=10, height=64, width=128, title=""):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        ax = plt.subplot(2,n,i+1)
        plt.imshow(X[i].reshape((height,width)))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.suptitle(title, fontsize = 20)

In [ ]:
show_car_data(x_test, title="original cats")
show_data(encoded_cat, title="Encoded")
show_car_data(reconstructed, title="reconstructed cats")